In [1]:
%matplotlib inline
%pylab inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
from __future__ import division
from mhcflurry import predict
from mhcflurry import class1_allele_specific
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

/home/vts/anaconda3/envs/python2/lib/python2.7/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


Populating the interactive namespace from numpy and matplotlib


Using Theano backend.
/home/vts/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [27]:
filepath = "./data/HIV_peptides.csv"
if os.path.isfile(filepath):
    df = pd.read_csv(filepath, sep=",")
    print("Reading {}".format(os.path.basename(filepath)))
else:
    print("File path is wrong")

Reading HIV_peptides.csv


In [28]:
df = df.dropna()
df = df.drop('Unnamed: 0', 1)
df.index = range(1,len(df) + 1)
print(len(df))
df.head(2)

1610


,binder,non_binder,mhc
1,GELDRWEKI,GELDRWkKI,B40
2,KIRLRPGGK,KIRLRPGGq,A3


In [29]:
df["non_binder"].iloc[0]

'GELDRWkKI'

In [30]:
#wdf = pd.DataFrame(columns=("Peptides", "MHC", "Binding"))
rows_list = []
for i in range(len(df)):
    dict1 = {}
    dict2 = {}
    dict1.update({'Peptide' : df['binder'].iloc[i], 'MHC' : df['mhc'].iloc[i], 'Binding' : 'Binder'})
    dict2.update({'Peptide' : df['non_binder'].iloc[i], 'MHC' : df['mhc'].iloc[i], 'Binding' : 'Non_binder'})
    rows_list.append(dict1)
    rows_list.append(dict2)
wdf = pd.DataFrame(rows_list)
print(len(wdf))
wdf['Peptide'] = wdf['Peptide'].str.upper()
wdf.head()

3220


,Binding,MHC,Peptide
0,Binder,B40,GELDRWEKI
1,Non_binder,B40,GELDRWKKI
2,Binder,A3,KIRLRPGGK
3,Non_binder,A3,KIRLRPGGQ
4,Binder,A3,KIRLRPGGK


In [31]:
print(unique(wdf['MHC']))

['A*01' 'A*0101' 'A*02' 'A*02, A*03' 'A*0201' 'A*0201, A*3002' 'A*0201, A2'
 'A*0202, A2' 'A*0205' 'A*0207' 'A*03' 'A*03, A*11' 'A*0301' 'A*0301, A*11'
 'A*0301, A*1101, A3, A*6801' 'A*0301, A*6801' 'A*11' 'A*1101' 'A*23'
 'A*23, A*2402, B*0801' 'A*23, A24' 'A*24' 'A*2402' 'A*2501' 'A*2601'
 'A*2601, A*2603' 'A*2603' 'A*2902' 'A*3002' 'A*31' 'A*3101' 'A*3201'
 'A*33' 'A*6801' 'A*6802' 'A1' 'A11' 'A11, A3' 'A2' 'A2, A3' 'A2, B62'
 'A2, B8' 'A2.1' 'A24' 'A24, B*5101' 'A24, B35' 'A25' 'A26' 'A26, B70'
 'A29' 'A3' 'A3 supertype, A*6801' 'A3, A30, B42, B62' 'A3, B7' 'A3, Cw3'
 'A30' 'A32, B14' 'A3?' 'B*07' 'B*0702' 'B*08' 'B*0801' 'B*13' 'B*1401'
 'B*1402' 'B*15' 'B*1501' 'B*1503' 'B*1510' 'B*1510, B70' 'B*1801' 'B*27'
 'B*2705' 'B*35' 'B*3501' 'B*37' 'B*3801' 'B*40' 'B*4001' 'B*4001, B50'
 'B*4201' 'B*4201, B*4202' 'B*4202' 'B*4402' 'B*4402, B*4403' 'B*4403'
 'B*4501' 'B*4801' 'B*51' 'B*5101' 'B*5401' 'B*57' 'B*57, B*58'
 'B*57, B*5801' 'B*5701' 'B*5701, B*5703' 'B*5701, B*58' 'B*5701, B*5

In [32]:
def f(row):
    if len(row['MHC']) == 6 and ',' not in row['MHC']:
        return 'HLA-{}'.format(row['MHC'])

wdf['MHC'] = wdf.apply(f, axis=1)

In [33]:
wdf = wdf.dropna()
wdf.head(5)

,Binding,MHC,Peptide
8,Binder,HLA-A*0301,KIRLRPGGK
9,Non_binder,HLA-A*0301,KIRLRPGGR
26,Binder,HLA-A*0301,RLRPGGKKK
27,Non_binder,HLA-A*0301,RLRPGGKKQ
42,Binder,HLA-A*0301,RLRPGGKKK


In [35]:
unique(wdf['MHC'])

array(['HLA-A*0101', 'HLA-A*0201', 'HLA-A*0205', 'HLA-A*0207',
       'HLA-A*0301', 'HLA-A*1101', 'HLA-A*2402', 'HLA-A*2501',
       'HLA-A*2601', 'HLA-A*2603', 'HLA-A*2902', 'HLA-A*3002',
       'HLA-A*3101', 'HLA-A*3201', 'HLA-A*6801', 'HLA-A*6802',
       'HLA-B*0702', 'HLA-B*0801', 'HLA-B*1401', 'HLA-B*1402',
       'HLA-B*1501', 'HLA-B*1503', 'HLA-B*1510', 'HLA-B*1801',
       'HLA-B*2705', 'HLA-B*3501', 'HLA-B*3801', 'HLA-B*4001',
       'HLA-B*4201', 'HLA-B*4202', 'HLA-B*4402', 'HLA-B*4403',
       'HLA-B*4501', 'HLA-B*4801', 'HLA-B*5101', 'HLA-B*5401',
       'HLA-B*5701', 'HLA-B*5703', 'HLA-B*5801', 'HLA-B*8101'], dtype=object)

In [39]:
prediction = predict(alleles='H-2<sup>d</sup>', peptides=wdf[wdf['MHC']=='H-2<sup>d</sup>']['Peptide'])

ValueError: No models for allele 'HLA-H'. Alleles with models: H-2-DB H-2-DD H-2-KB H-2-KD H-2-KK H-2-LD HLA-A0101 HLA-A0201 HLA-A0202 HLA-A0203 HLA-A0206 HLA-A0211 HLA-A0212 HLA-A0216 HLA-A0217 HLA-A0219 HLA-A0301 HLA-A1101 HLA-A2301 HLA-A2402 HLA-A2403 HLA-A2501 HLA-A2601 HLA-A2602 HLA-A2603 HLA-A2902 HLA-A3001 HLA-A3002 HLA-A3101 HLA-A3201 HLA-A3301 HLA-A6801 HLA-A6802 HLA-A6901 HLA-A8001 HLA-B0702 HLA-B0801 HLA-B0802 HLA-B0803 HLA-B1402 HLA-B1501 HLA-B1503 HLA-B1509 HLA-B1517 HLA-B1542 HLA-B1801 HLA-B2703 HLA-B2705 HLA-B3501 HLA-B3503 HLA-B3801 HLA-B3901 HLA-B4001 HLA-B4002 HLA-B4201 HLA-B4402 HLA-B4403 HLA-B4501 HLA-B4506 HLA-B4601 HLA-B4801 HLA-B5101 HLA-B5301 HLA-B5401 HLA-B5701 HLA-B5801 HLA-B8301 HLA-C0401 HLA-C0602 HLA-C1402 HLA-EQCA100101 Mamu-A01 Mamu-A02 Mamu-A07 Mamu-A100101 Mamu-A100201 Mamu-A101101 Mamu-A11 Mamu-A2201 Mamu-A2601 Mamu-B01 Mamu-B01704 Mamu-B03 Mamu-B08 Mamu-B17 Mamu-B3901 Mamu-B52 Mamu-B8301 Patr-A0101 Patr-A0301 Patr-A0401 Patr-A0701 Patr-A0901 Patr-B010

In [37]:
predictions = pd.DataFrame()
warning = pd.DataFrame()
warnings = 0
warning_loc = []
for i in range(len(wdf)):
    try:
        prediction = predict(alleles=wdf["MHC"].iloc[i:i+1].reset_index(drop=True),
                                   peptides=wdf["Peptide"].iloc[i:i+1].reset_index(drop=True))
        frame = [predictions, prediction]
        predictions = pd.concat(frame)
    except ValueError:
        warnings += 1
        frame = [warning, wdf.iloc[i]]
        warning = pd.concat(frame)
        warning_loc.append(i)
if len(predictions):
    print("Predictions made")
else:
    print("Something went wrong")
print("Number of warnings is {}".format(warnings))
print(warning)

Predictions made
Number of warnings is 66
                   0
Binding       Binder
MHC       HLA-B*5703
Peptide  KAFSPEVIPMF
Binding   Non_binder
MHC       HLA-B*5703
Peptide  KNFSPEVIPMF
Binding       Binder
MHC       HLA-B*5703
Peptide  KAFSPEVIPMF
Binding   Non_binder
MHC       HLA-B*5703
Peptide  RAFSPEVIPMF
Binding       Binder
MHC       HLA-B*4202
Peptide    TPQDLNMML
Binding   Non_binder
MHC       HLA-B*4202
Peptide    TPQDLNTML
Binding       Binder
MHC       HLA-B*8101
Peptide    TPQDLNTML
Binding   Non_binder
MHC       HLA-B*8101
Peptide    TPEDLNTML
Binding       Binder
MHC       HLA-B*8101
Peptide    TPQDLNTML
Binding   Non_binder
MHC       HLA-B*8101
Peptide    TPQDLNMML
...              ...
Binding       Binder
MHC       HLA-B*8101
Peptide    RPQVPLRPM
Binding   Non_binder
MHC       HLA-B*8101
Peptide    RPQVPVRPM
Binding       Binder
MHC       HLA-B*8101
Peptide    RPQVPLRPM
Binding   Non_binder
MHC       HLA-B*8101
Peptide    RPQVPTRPM
Binding       Binder
MHC       HLA

In [38]:
predictions.head(5)

,Allele,Peptide,Prediction
0,HLA-A0301,KIRLRPGGK,44.690437
0,HLA-A0301,KIRLRPGGR,542.003174
0,HLA-A0301,RLRPGGKKK,26.093128
0,HLA-A0301,RLRPGGKKQ,1990.704102
0,HLA-A0301,RLRPGGKKK,26.093128


In [39]:
def coding(col, codeDict):
    colCoded = pd.Series(col, copy=True)
    for key, value in codeDict.items():
        colCoded.replace(key, value, inplace=True)
    return colCoded

In [40]:
true_values = coding(wdf["Binding"], {"Binder":1, "Non_binder":0})
true_values = pd.DataFrame(true_values)
true_values.index = range(1,len(true_values) + 1)
true_values.drop(true_values.index[warning_loc], inplace=True)
true_values["Binding"].value_counts()

1    584
0    584
Name: Binding, dtype: int64

In [41]:
predicted_values = predictions["Prediction"]
predicted_values.index = range(1,len(predicted_values) + 1)
predicted_values.columns = ["Prediction"]
predicted_values = pd.DataFrame(predicted_values)
predicted_values["Prediction"] = np.where(predicted_values["Prediction"] <= 500, 1, 0)
predicted_values["Prediction"].value_counts()

1    764
0    404
Name: Prediction, dtype: int64

In [42]:
accuracy_score(true_values, predicted_values)

0.56849315068493156

In [43]:
roc_auc_score(true_values, predicted_values)

0.56849315068493145

In [44]:
confusion_matrix(true_values, predicted_values, labels=[0,1])

array([[242, 342],
       [162, 422]])

In [57]:
tn, fp, fn, tp = confusion_matrix(true_values, predicted_values, labels=[0,1]).ravel()
ln = len(true_values)
print(' true negative   {0:.2} \n \
false positive  {1:.2} \n \
false negative  {2:.2} \n \
true positive   {3:.2}'.format(tn/ln, fp/ln, fn/ln, tp/ln))

 true negative   0.21 
 false positive  0.29 
 false negative  0.14 
 true positive   0.36
